In [2]:
# read data

from mnist import MNIST
import numpy as np
mndata = MNIST("/Users/jigyayadav/Desktop/Codes/neuralnets253/HW1")
mndata.gz = True
images, labels = mndata.load_training() #Images is a list of 60000 images of 784 dimensions, Labels is a list of 60000 ints
imagesTest, labelsTest = mndata.load_testing()

In [3]:
images = np.array(images)
labels = np.array(labels)
imagesTest = np.array(imagesTest)
labelsTest = np.array(labelsTest)

images_train = images[:2000]
images_test = imagesTest[-50:]
labels_train = labels[:2000]
labels_test = labelsTest[-50:]
images_train = np.array(images_train)
labels_train = np.array(labels_train)
images_test = np.array(images_test)
labels_test = np.array(labels_test)

# Normalization
images_train = images_train*(1/127.5)
images_test = images_test*(1/127.5)
images_train = images_train-1.0
images_test = images_test-1.0

# Should this be appended after or before normalization
images_train = np.insert(images_train, 0, 1, axis=1)
images_test = np.insert(images_test, 0, 1, axis=1)

numFeatures = len(images[0, :])

# Divide between validation and training
from sklearn.model_selection import train_test_split
images_train, images_validation, labels_train, labels_validation = train_test_split(images_train, labels_train, test_size=0.10)

In [4]:
labelNums = 10

def labelsToVectForm(labels):
    vectForm = np.zeros((len(labels), labelNums))
    for i in range(len(labels)):
        vectForm[i][labels[i]] = 1
    return vectForm

t_train = labelsToVectForm(labels_train)
t_test = labelsToVectForm(labels_test)
t_validation = labelsToVectForm(labels_validation)

x = images_train
t = t_train

In [5]:
images_validation.shape

(20, 785)

In [7]:
from sklearn.metrics import accuracy_score
import math

# Forward propagation

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmaxFunc(prod):
    prod = np.exp(prod)
    for i in range(len(prod)):
        rowSum = sum(prod[i, :])
        for j in range(len(prod[i, :])):
            prod[i][j] = prod[i][j]/rowSum    
    return prod

def getPredClass(y):
    predClass = np.argmax(y, axis=1)
    return predClass

def calculateOutput(x, W1, W2):
    A1 = x.dot(W1)
    Z1 = sigmoid(A1)
    X2 = np.insert(Z1, 0, 1, axis=1)
    A2 = X2.dot(W2)
    Y = softmaxFunc(A2)
    return Y

def calculateCost(x, t, W1, W2):
    sampLen = len(x)
    Y = calculateOutput(x, W1, W2)
    cost = -(1/len(x))*(np.sum(np.multiply(t, np.log(Y))))
    return cost

def calculateCostOneInstance(x, t, W1, W2, n):
    sampLen = len(x)
    Y = calculateOutput(x, W1, W2)
    cost = -1*(np.multiply(t, np.log(Y)))
    return np.sum(cost[n, :])

def calculateAccuracy(x, labels, W1, W2):
    Y = calculateOutput(x, W1, W2)
    predClass = getPredClass(Y)
    return accuracy_score(labels, predClass)


W1 = np.random.uniform(low=-1.0, high=1.0, size=(785, 64))
W2 = np.random.uniform(low=-1.0, high=1.0, size=(65, 10))
epsilon = 0.001
learningRate = 0.01

batch_size = 128
batches = x.shape[0]//batch_size

for itr in range(20000):
    for b in range(batches):
        x = 
        A1 = x.dot(W1)
        Z1 = sigmoid(A1)

    #     print(Z1[0, :])
        X2 = np.insert(Z1, 0, 1, axis=1)
    #     print(X2[0, :])
    #     print("X2 : ")
    #     print(X2)
    #     print("W2 : ")
    #     print(W2)
        A2 = X2.dot(W2)
    #     print("A2 : ")
    #     print(A2)
    #     exp_Y = np.exp(A2)
        Y = np.exp(A2)/np.sum(np.exp(A2), axis=1, keepdims = True)

    #     Y_p = softmaxFunc(A2)
    # #     print("Y-Y_p")
    # #     print(Y-Y_p)

        # Backward propagation
        W2_nobias = np.delete(W2, (0), axis=0)
        summation_k = (Y-t).dot(W2_nobias.T)
        g_derivative = Z1*(1-Z1)
        delE_Wij = x.T.dot(np.multiply(summation_k, g_derivative))
        delE_Wij = delE_Wij

        # Check approximate derivative
    #     for n in range(0, 150, 30):
    #         print("For input to hidden layers")
    #         for i in range(0, 785, 100):
    #             for j in range(0, 64, 20):
    #                 del_En_Wij = (g_derivative[n][j]*summation_k[n][j]*x[n][i])
    #                 modifiedW1 = W1
    #                 modifiedW1[i][j] += epsilon
    #                 costFunction1 = calculateCostOneInstance(x, t, modifiedW1, W2, n)
    #                 modifiedW1[i][j] -= 2*epsilon
    #                 costFunction2 = calculateCostOneInstance(x, t, modifiedW1, W2, n)
    #                 approx_delEn = (costFunction1-costFunction2)/(2*epsilon)
    #                 diff = abs(del_En_Wij-approx_delEn)
    # #                 print(del_En_Wij, " ", approx_delEn)
    #                 if (diff > epsilon*epsilon):
    #                     print("Difference between approximate gradient = ", diff, " ", abs(epsilon*epsilon-diff))

    #         print("For hidden to output layers")
    #         for j in range(0, 65, 20):
    #             for k in range(0, 10, 3):
    #                 actualDerivative = X2[n][j]*(Y[n][k]-t[n][k])
    #                 modifiedW2 = W2
    #                 modifiedW2[j][k] += epsilon
    #                 costFunction1 = calculateCostOneInstance(x, t, W1, modifiedW2, n)
    #                 modifiedW2[j][k] -= 2*epsilon
    #                 costFunction2 = calculateCostOneInstance(x, t, W1, modifiedW2, n)
    #                 approximateDerivative = (costFunction1-costFunction2)/(2*epsilon)
    #                 diff = abs(actualDerivative-approximateDerivative)
    # #                 print(actualDerivative, " ", approximateDerivative)
    #                 if (diff > epsilon*epsilon):
    #                     print("Difference between approximate gradient = ", diff, " ", abs(epsilon*epsilon-diff))

        W2 = W2-(learningRate/len(x))*((X2.T.dot(Y-t)))
        W1 = W1-(learningRate/len(x))*(delE_Wij)

        # Calculate validation error
        currTrainingError = calculateCost(x, t, W1, W2)
        currvalidationError = calculateCost(images_validation, t_validation, W1, W2)
        currTrainingAccuracy = calculateAccuracy(x, labels_train, W1, W2)
        currValidationAccuracy = calculateAccuracy(images_validation, labels_validation, W1, W2)
        print("Current training error = ", currTrainingError)
        print("Current validation error = ", currvalidationError)
        print("Current training accuracy = ", currTrainingAccuracy)
        print("Current validation accuracy = ", currValidationAccuracy)
    

Current training error =  5.068445674866948
Current validation error =  4.457186191465428
Current training accuracy =  0.12222222222222222
Current validation accuracy =  0.2
Current training error =  4.99910957536519
Current validation error =  4.406287094439391
Current training accuracy =  0.12222222222222222
Current validation accuracy =  0.2
Current training error =  4.931662584879424
Current validation error =  4.357138685319888
Current training accuracy =  0.12777777777777777
Current validation accuracy =  0.2
Current training error =  4.866054091224747
Current validation error =  4.309623983829033
Current training accuracy =  0.13333333333333333
Current validation accuracy =  0.2
Current training error =  4.80223597436604
Current validation error =  4.263636435924221
Current training accuracy =  0.1388888888888889
Current validation accuracy =  0.2
Current training error =  4.740160939886438
Current validation error =  4.219078807676034
Current training accuracy =  0.138888888888

Current training error =  2.8747568455436356
Current validation error =  2.947186058308249
Current training accuracy =  0.2222222222222222
Current validation accuracy =  0.2
Current training error =  2.859034662272059
Current validation error =  2.938262085188107
Current training accuracy =  0.2222222222222222
Current validation accuracy =  0.2
Current training error =  2.843643151364491
Current validation error =  2.9295671907256984
Current training accuracy =  0.2222222222222222
Current validation accuracy =  0.2
Current training error =  2.828576219908267
Current validation error =  2.921091721895484
Current training accuracy =  0.2222222222222222
Current validation accuracy =  0.2
Current training error =  2.8138279892912603
Current validation error =  2.912826561013408
Current training accuracy =  0.22777777777777777
Current validation accuracy =  0.2
Current training error =  2.799392717678168
Current validation error =  2.904763113189753
Current training accuracy =  0.2277777777

Current training error =  2.3396872860462823
Current validation error =  2.646559693674577
Current training accuracy =  0.3333333333333333
Current validation accuracy =  0.15
Current training error =  2.3338258207887814
Current validation error =  2.643123330423439
Current training accuracy =  0.3333333333333333
Current validation accuracy =  0.15
Current training error =  2.3280277841445627
Current validation error =  2.6397177255724866
Current training accuracy =  0.3333333333333333
Current validation accuracy =  0.15
Current training error =  2.322291354629648
Current validation error =  2.636341914736504
Current training accuracy =  0.3333333333333333
Current validation accuracy =  0.15
Current training error =  2.316614778361819
Current validation error =  2.6329949728751822
Current training accuracy =  0.3333333333333333
Current validation accuracy =  0.15
Current training error =  2.3109963661629433
Current validation error =  2.6296760126460983
Current training accuracy =  0.33

Current training error =  2.0799731681100853
Current validation error =  2.4875462988023322
Current training accuracy =  0.36666666666666664
Current validation accuracy =  0.15
Current training error =  2.0760536231720033
Current validation error =  2.4850583138047857
Current training accuracy =  0.36666666666666664
Current validation accuracy =  0.15
Current training error =  2.0721554438999465
Current validation error =  2.482582800719708
Current training accuracy =  0.36666666666666664
Current validation accuracy =  0.15
Current training error =  2.0682785102962065
Current validation error =  2.480119780180882
Current training accuracy =  0.36666666666666664
Current validation accuracy =  0.15
Current training error =  2.0644227125338808
Current validation error =  2.4776692700721856
Current training accuracy =  0.36666666666666664
Current validation accuracy =  0.15
Current training error =  2.060587950002457
Current validation error =  2.4752312843040896
Current training accuracy 

Current training error =  1.8707471870864794
Current validation error =  2.3517420446161017
Current training accuracy =  0.40555555555555556
Current validation accuracy =  0.15
Current training error =  1.8679509965688523
Current validation error =  2.3498311799002494
Current training accuracy =  0.40555555555555556
Current validation accuracy =  0.15
Current training error =  1.8651678084545045
Current validation error =  2.34792526073113
Current training accuracy =  0.4111111111111111
Current validation accuracy =  0.15
Current training error =  1.862397497209357
Current validation error =  2.346024233855896
Current training accuracy =  0.4111111111111111
Current validation accuracy =  0.15
Current training error =  1.8596399408949662
Current validation error =  2.3441280503817397
Current training accuracy =  0.4111111111111111
Current validation accuracy =  0.15
Current training error =  1.8568950210725939
Current validation error =  2.342236665680813
Current training accuracy =  0.

...

Current training error =  0.08849623383862604
Current validation error =  0.9863600228727972
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.08848238915187802
Current validation error =  0.986346986084088
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.08846854827538447
Current validation error =  0.9863339515926077
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.08845471120803879
Current validation error =  0.9863209193960127
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.0884408779487344
Current validation error =  0.986307889491966
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.08842704849636487
Current validation error =  0.9862948618781449
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.088413222

Current training error =  0.08779498987263915
Current validation error =  0.9856980292916652
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.08778133810030138
Current validation error =  0.9856851068803323
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.08776769008191983
Current validation error =  0.9856721866593292
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.08775404581639296
Current validation error =  0.9856592686268091
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.08774040530261928
Current validation error =  0.9856463527809401
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.08772676853949747
Current validation error =  0.9856334391198956
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.087713

Current training error =  0.08710351285274723
Current validation error =  0.9850417428907108
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.08709005112889219
Current validation error =  0.9850289300264099
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.08707659310288217
Current validation error =  0.9850161192700018
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.08706313877361788
Current validation error =  0.9850033106201006
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.08704968814000015
Current validation error =  0.9849905040753255
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.08703624120092979
Current validation error =  0.9849776996343049
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.087022

Current training error =  0.08632881149620576
Current validation error =  0.9843020417618306
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.08631556243079906
Current validation error =  0.984289349112339
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.0863023169995024
Current validation error =  0.98427665850354
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.08628907520121708
Current validation error =  0.9842639699344989
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.08627583703484425
Current validation error =  0.9842512834042911
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.08626260249928508
Current validation error =  0.9842385989119983
Current training accuracy =  1.0
Current validation accuracy =  0.7
Current training error =  0.0862493715

KeyboardInterrupt: 